# Understanding Agentic AI Frameworks

## What is Agentic AI?

Agentic AI refers to autonomous systems that can perceive their environment, make decisions, and take actions to achieve specific goals. Unlike traditional LLM applications that passively respond to prompts, agents actively reason about problems and iteratively refine their approach.

## Key Characteristics of Agents

- **Autonomy**: Agents operate independently with minimal human intervention
- **Reasoning**: They break down complex problems into manageable steps
- **Tool Usage**: Agents leverage external tools (search engines, calculators, APIs) to augment their capabilities
- **Iterative Refinement**: They can loop through thought-action-observation cycles to improve results
- **Adaptability**: Agents can adjust their strategy based on feedback from their actions

## Why Agentic Frameworks Matter

1. **Complex Problem Solving**: Agents can tackle multi-step problems that require planning and tool usage
2. **Real-time Information Access**: By using tools, agents can retrieve current information beyond their training data
3. **Transparent Reasoning**: The thought-action-observation process provides interpretability
4. **Cost Efficiency**: By routing tasks intelligently, agents can reduce unnecessary API calls
5. **Scalability**: Agents can handle a wider variety of tasks without being retrained

This notebook demonstrates a **Simple ReAct Agent** - one of the most effective agentic frameworks that combines Reasoning and Acting in a structured loop.

<h1>Simple Agent</h1>


This exampl is taken from Chapter 7 of the [Hands-On Large Language Models](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961) book by [Jay Alammar](https://www.linkedin.com/in/jalammar) and [Maarten Grootendorst](https://www.linkedin.com/in/mgrootendorst/).



## Notebook Overview

This notebook implements a **ReAct Agent** based on the foundational concepts from the "Hands-On Large Language Models" book. The ReAct (Reasoning + Acting) framework is one of the most effective approaches for building AI agents that can solve complex, multi-step problems by:

1. **Reasoning** about what needs to be done
2. **Acting** by calling relevant tools
3. **Observing** the results
4. **Iterating** until the final answer is reached

This practical example shows how to integrate LLMs with external tools to create a truly intelligent agent.

In [ ]:
%%capture
!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community


## Setting Up Dependencies

To build our agent, we need several key libraries:

- **LangChain**: A framework for building applications with language models, providing abstractions for agents, tools, and chains
- **OpenAI**: The language model provider (GPT-3.5-turbo in this case)
- **DuckDuckGo Search**: A tool for real-time web search, allowing our agent to access current information
- **Sentence Transformers**: For semantic understanding and embeddings
- **Transformers & Datasets**: Core ML/AI libraries for working with pre-trained models

These dependencies enable our agent to not only reason but also act by accessing external information sources and performing computations.

# Agents

## Core Concepts: What Are Agents?

An **Agent** is an AI system that:

1. **Perceives**: Receives input and understands the task
2. **Thinks**: Uses an LLM to reason about what steps are needed
3. **Acts**: Selects and executes appropriate tools
4. **Observes**: Reviews the results of its actions
5. **Iterates**: Repeats the cycle until it reaches the final answer

### Agent Components

- **Language Model**: The "brain" that decides what to do (GPT-3.5-turbo)
- **Tools**: External capabilities (search, math, APIs, databases)
- **Prompt Template**: Instructs the LLM how to think and act
- **Agent Executor**: Orchestrates the reasoning loop

The key innovation of agents is that they can **chain thoughts and actions together**, making them capable of solving problems that require multiple steps and tool interactions.

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults
from google.colab import userdata

# Load OpenAI's LLMs with LangChain
os.environ["OPENAI_API_KEY"] = userdata.get('OPEN_AI_KEY')
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANG_KEY')
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "simple_agent_book_example"


## Configuring the Language Model and Environment

Here we initialize:

1. **ChatOpenAI Model**: We use GPT-3.5-turbo with `temperature=0` for deterministic, focused responses
2. **API Keys**: Secure configuration using environment variables
3. **LangChain Tracing**: Enables monitoring and debugging of agent execution in LangSmith

**Why These Choices Matter:**
- **Temperature=0**: Ensures consistent, reproducible responses from the LLM
- **GPT-3.5-turbo**: Balances performance and cost for agent applications
- **LangSmith Integration**: Provides visibility into what the agent is thinking and doing (crucial for debugging and improvement)

In [ ]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

## The ReAct Framework: Reasoning + Acting

The **ReAct** (Reasoning + Acting) framework is a structured approach that makes agents more effective:

### The ReAct Loop Structure

```
Thought → Action → Observation → Thought → ... → Final Answer
```

### Why ReAct Works So Well

1. **Explainability**: Each step is transparent and interpretable
2. **Error Correction**: The agent can observe failures and adjust its approach
3. **Composability**: Multiple tools can be chained together
4. **Robustness**: The structured format reduces hallucinations and nonsensical outputs

### How the Prompt Template Works

The prompt instructs the LLM to:
- **Thought**: Reason about the problem and what tool to use
- **Action**: Specify which tool to call
- **Action Input**: Provide parameters for the tool
- **Observation**: Show what the tool returned
- **Repeat**: Continue until "Final Answer" is reached

This explicit structure guides the LLM's reasoning process, making it more reliable and interpretable than free-form responses.

In [ ]:
# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

## Building the Agent's Toolkit

An agent's effectiveness depends on the tools available to it. Here we create:

### Types of Tools

1. **Search Tool (DuckDuckGo)**: 
   - Retrieves real-time information from the web
   - Allows the agent to answer questions beyond its training data
   - Essential for current events, prices, and dynamic information

2. **Math Tool (llm-math)**:
   - Performs accurate mathematical calculations
   - Addresses LLM weakness in precise arithmetic
   - Can handle complex expressions

### Why Tool Design Matters

- **Specificity**: Each tool should have a clear, single purpose
- **Descriptions**: Detailed tool descriptions help the agent decide when to use them
- **Reliability**: Tools must return consistent, accurate results
- **Composability**: Tools can be combined to solve complex problems

The agent will automatically choose which tools to use based on the task, demonstrating true reasoning ability.

In [ ]:
%pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.0 MB/s eta 0:00:00


## Dependency Update

Installing or updating the `duckduckgo-search` library to ensure optimal performance of the search tool.

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

## Creating the Agent Executor

The **AgentExecutor** is the orchestrator that brings everything together:

### Key Components

1. **Agent**: Implements the reasoning logic (in this case, ReAct)
2. **Tools**: The capabilities available to the agent
3. **Executor**: Manages the loop and handles errors

### Important Parameters

- **verbose=True**: Prints out the agent's thought process (crucial for understanding and debugging)
- **handle_parsing_errors=True**: Gracefully handles cases where the LLM doesn't follow the exact format

### How It Works

The executor:
1. Sends the input to the agent
2. The LLM generates thoughts and decides on actions
3. Executor calls the specified tool with the action input
4. Result is fed back to the LLM
5. Process repeats until "Final Answer" is generated

This loop continues until the agent produces a final answer or reaches a termination condition.

In [ ]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
I should use a web search engine to find the current price of a MacBook Pro in USD and then use a calculator to convert it to EUR based on the exchange rate.
Action: duckduck
Action Input: "current price of MacBook Pro in USD"snippet: May 2, 2025 - Microcenter has the 36GB version for $1699 if you're in search of more RAM. https://www.microcenter.com/product/692052/apple-macbook-pro-14-z1au0029n-(late-2023)-142-laptop-computer-space-black ..., title: Reddit r/buildapcsales on Reddit: [Laptop] Apple 14" MacBook Pro (M3 Pro, Space Black) - $1399 ($1999 - $600) (B&H Photo Video), link: https://www.reddit.com/r/buildapcsales/comments/1kd0d06/laptop_apple_14_macbook_pro_m3_pro_space_black/, snippet: 1 week ago - If a big screen is your top priority, you can score a $264 discount on the 16-inch model of the latest MacBook Pro, dropping the price to $2,235 on Amazon . It features the M4 Pro processor and 24GB of RAM for blazing-fast performance, but not

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?',
 'output': 'The current price of a MacBook Pro in USD is $1,399. It would cost approximately 1189.15 EUR based on the exchange rate of 0.85 EUR for 1 USD.'}

## Executing the Agent: Putting It All Together

Now we test our agent with a real-world multi-step problem:

### The Challenge

The question requires:
1. **Web Search**: Finding the current price of a MacBook Pro
2. **Math Calculation**: Converting from USD to EUR using a given exchange rate
3. **Reasoning**: Connecting these pieces to provide a complete answer

### What to Observe

When you run this cell, watch for:
- **Thought**: The agent's reasoning about what to do
- **Action**: Which tool it chooses (search or calculator)
- **Observation**: The results returned by each tool
- **Final Answer**: The synthesized answer combining multiple tool results

This demonstrates the true power of agents: solving problems that require **combining multiple tools and reasoning steps** - something that a simple LLM cannot do alone.